# Car Price Data Exploration

This notebook explores the car price data from Le Bon Coin using Polars for efficient data processing. The data is stored in multiple CSV files in `/Users/brunobrumbrum/Documents/data/le_boncoin_13_oct_2025`.

## Objectives:
- Load and explore all CSV files from the data directory
- Analyze data structure, quality, and patterns
- Generate visualizations for better understanding
- Optimize memory usage with Polars

## 1. Setup Environment and Imports

Import necessary libraries including Polars, matplotlib, seaborn, and pathlib for data processing and visualization.

In [1]:
# Core data processing libraries
import polars as pl
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Path and file handling
from pathlib import Path
import glob
import os
import sys

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

# Display options for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Add project root to Python path for module imports
current_dir = Path.cwd()
project_root = current_dir.parent if current_dir.name == "notebooks" else current_dir
sys.path.insert(0, str(project_root))

from src.config import DATA_PATH, PROCESSED_DATA_PATH, MODELS_PATH

## 3. Read CSV Files from Data Directory

Use Polars to read all CSV files from the specified directory with `infer_schema_length=0` for faster loading.

In [ ]:
# Define data directory
data_dir = Path(os.path.join(DATA_PATH, "le_boncoin_13_oct_2025"))
# Find all CSV files
csv_files = list(data_dir.glob("*.csv"))

In [ ]:
# Read all CSV files using Polars with fast loading
dataframes = {}
total_rows = 0

print("📥 Loading CSV files with Polars...")
print("-" * 50)

for file_path in csv_files:
    try:
        df = pl.read_csv(
            file_path,
            infer_schema_length=0,
            #encoding="utf8",
        )
        
        dataframes[file_path.stem] = df
        rows = df.height
        cols = df.width
        total_rows += rows
        
        print(f"✅ {file_path.name}: {rows:,} rows × {cols} columns")
        
    except Exception as e:
        print(f"❌ Error loading {file_path.name}: {e}")

print("-" * 50)
print(f"📊 Total datasets loaded: {len(dataframes)}")
print(f"📊 Total rows across all files: {total_rows:,}")

# Show memory usage
memory_usage = sum(df.estimated_size("mb") for df in dataframes.values())
print(f"💾 Estimated memory usage: {memory_usage:.2f} MB")

📥 Loading CSV files with Polars...
--------------------------------------------------
✅ DELAGE.csv: 4 rows × 35 columns
✅ GENERAL MOTORS.csv: 71 rows × 35 columns
✅ AUSTIN.csv: 282 rows × 35 columns
✅ PIAGGIO.csv: 18 rows × 35 columns
✅ MERCURY.csv: 18 rows × 35 columns
✅ LINCOLN.csv: 39 rows × 35 columns
✅ TRABANT.csv: 5 rows × 35 columns
✅ DAF.csv: 8 rows × 35 columns
✅ LAMBORGHINI.csv: 278 rows × 35 columns
✅ BERTONE.csv: 6 rows × 35 columns
✅ DACIA.csv: 18,670 rows × 35 columns
✅ SUZUKI.csv: 4,668 rows × 35 columns
✅ CASALINI.csv: 79 rows × 35 columns
✅ OPEL.csv: 19,869 rows × 35 columns
✅ LEAPMOTOR.csv: 146 rows × 35 columns
✅ GENESIS.csv: 2 rows × 35 columns
✅ ALFA ROMEO.csv: 4,539 rows × 35 columns
✅ BYD.csv: 434 rows × 35 columns
✅ PORSCHE.csv: 10,838 rows × 35 columns
✅ MORGAN.csv: 143 rows × 35 columns
✅ KIA.csv: 10,345 rows × 35 columns
✅ BMW.csv: 48,326 rows × 35 columns
✅ UMM.csv: 4 rows × 35 columns
✅ HUMMER.csv: 92 rows × 35 columns
✅ SILENCE.csv: 3 rows × 35 columns
✅ H

## 4. Data Schema and Structure Analysis

Examine the schema, column names, data types, and structure of the datasets.

In [16]:
df = pl.concat(dataframes.values(), how="vertical")

print(df.shape)
df.head(3)

(732427, 35)


url,first_publication_date,index_date,price,marque,modele,annee_modele,kilometrage,energie,boite_de_vitesse,nombre_de_portes,nombre_de_place_s,version_constructeur,date_de_premiere_mise_en_circulation,type_de_vehicule,couleur,crit_air,puissance_fiscale,puissance_din,permis,reference,duree_de_disponibilite_des_pieces_detachees,pays,id_region,region,id_departement,departement,ville_affichee,ville,code_postal,latitude,longitude,source,fournisseur,forme_existante
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""https://www.leboncoin.fr/ad/vo…","""2025-09-01 15:37:26""","""2025-10-08 18:28:10""","""24900 €""","""DELAGE""","""D4""","""1960""","""15000 km""","""Essence""","""Manuelle""","""4""","""4""",null,"""01/1935""","""Berline""","""Beige""",null,"""8 Cv""",null,"""Avec permis""",null,null,"""FR""","""3""","""Auvergne""","""63""","""Puy-de-Dôme""","""Romagnat 63540 Opme""","""Romagnat""","""63540""","""45.70758""","""3.08908""","""city""","""here""","""True"""
"""https://www.leboncoin.fr/ad/vo…","""2025-10-06 19:02:42""","""2025-10-06 19:02:42""","""39000 €""","""DELAGE""","""DI""","""1960""","""43000 km""","""Essence""","""Manuelle""","""4""","""6""",null,"""01/1924""","""Cabriolet""",null,null,"""11 Cv""","""35 Ch""","""Avec permis""",null,null,"""FR""","""17""","""Nord-Pas-de-Calais""","""62""","""Pas-de-Calais""","""La Couture 62136""","""La Couture""","""62136""","""50.58426""","""2.70473""","""city""","""here""","""True"""
"""https://www.leboncoin.fr/ad/vo…","""2025-09-30 20:16:40""","""2025-09-30 20:16:40""","""18000 €""","""DELAGE""","""DI""","""1960""","""120000 km""","""Essence""","""Manuelle""","""4""","""5""",null,"""01/1936""","""Berline""","""Bleu""",null,"""12 Cv""","""80 Ch""","""Avec permis""",null,null,"""FR""","""17""","""Nord-Pas-de-Calais""","""59""","""Nord""","""Rousies 59131""","""Rousies""","""59131""","""50.27274""","""4.00581""","""city""","""here""","""True"""


In [17]:
df.columns

['url',
 'first_publication_date',
 'index_date',
 'price',
 'marque',
 'modele',
 'annee_modele',
 'kilometrage',
 'energie',
 'boite_de_vitesse',
 'nombre_de_portes',
 'nombre_de_place_s',
 'version_constructeur',
 'date_de_premiere_mise_en_circulation',
 'type_de_vehicule',
 'couleur',
 'crit_air',
 'puissance_fiscale',
 'puissance_din',
 'permis',
 'reference',
 'duree_de_disponibilite_des_pieces_detachees',
 'pays',
 'id_region',
 'region',
 'id_departement',
 'departement',
 'ville_affichee',
 'ville',
 'code_postal',
 'latitude',
 'longitude',
 'source',
 'fournisseur',
 'forme_existante']

## 6. Data Quality Assessment

Check for missing values, duplicates, data type inconsistencies, and other data quality issues.

In [ ]:
print("🔍 DATA QUALITY ASSESSMENT")
print("=" * 50)

quality_report = []

for name, df in dataframes.items():
    # Basic quality metrics
    total_cells = df.height * df.width
    null_cells = df.null_count().sum(axis=1)[0]
    null_percentage = (null_cells / total_cells) * 100
    
    duplicate_rows = df.height - df.unique().height
    duplicate_percentage = (duplicate_rows / df.height) * 100
    
    # Check for completely empty columns
    empty_columns = [col for col in df.columns if df[col].null_count()[0] == df.height]
    
    quality = {
        'Dataset': name,
        'Completeness (%)': f"{100 - null_percentage:.1f}",
        'Uniqueness (%)': f"{100 - duplicate_percentage:.1f}",
        'Empty Columns': len(empty_columns),
        'Quality Score': f"{max(0, 100 - null_percentage - duplicate_percentage):.1f}"
    }
    quality_report.append(quality)
    
    if empty_columns:
        print(f"⚠️  {name} has {len(empty_columns)} completely empty columns: {empty_columns}")

# Display quality report
quality_df = pd.DataFrame(quality_report)
print("\n📊 QUALITY REPORT:")
print(quality_df.to_string(index=False))

In [ ]:
# Sample data inspection for potential car price columns
print("🚗 CAR PRICE RELATED COLUMNS DETECTION")
print("=" * 50)

# Look for price-related columns
price_keywords = ['price', 'prix', 'cost', 'value', 'montant', 'tarif']
car_keywords = ['car', 'auto', 'vehicle', 'voiture', 'marque', 'modele', 'model', 'brand']

for name, df in dataframes.items():
    print(f"\n📋 Dataset: {name}")
    
    # Find potential price columns
    price_cols = [col for col in df.columns 
                  if any(keyword in col.lower() for keyword in price_keywords)]
    
    # Find potential car-related columns
    car_cols = [col for col in df.columns 
                if any(keyword in col.lower() for keyword in car_keywords)]
    
    if price_cols:
        print(f"  💰 Potential price columns: {price_cols}")
        # Show sample values for price columns
        for col in price_cols[:3]:  # Limit to first 3
            sample_values = df[col].drop_nulls().head(5).to_list()
            print(f"     {col} samples: {sample_values}")
    
    if car_cols:
        print(f"  🚗 Potential car columns: {car_cols}")
        # Show sample values for car columns
        for col in car_cols[:3]:  # Limit to first 3
            sample_values = df[col].drop_nulls().head(5).to_list()
            print(f"     {col} samples: {sample_values}")
    
    if not price_cols and not car_cols:
        print("  ❓ No obvious price or car columns detected")
        print(f"     All columns: {df.columns[:10]}{'...' if len(df.columns) > 10 else ''}")

## 7. Visual Data Exploration

Create visualizations including histograms, box plots, correlation matrices, and distribution plots.

In [ ]:
# Visualize dataset sizes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Dataset sizes
dataset_names = list(dataframes.keys())
dataset_sizes = [df.height for df in dataframes.values()]

ax1.bar(range(len(dataset_names)), dataset_sizes, color=sns.color_palette("husl", len(dataset_names)))
ax1.set_title('Dataset Sizes (Number of Rows)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Datasets')
ax1.set_ylabel('Number of Rows')
ax1.set_xticks(range(len(dataset_names)))
ax1.set_xticklabels(dataset_names, rotation=45, ha='right')

# Add value labels on bars
for i, v in enumerate(dataset_sizes):
    ax1.text(i, v + max(dataset_sizes) * 0.01, f'{v:,}', 
             ha='center', va='bottom', fontweight='bold')

# Memory usage
memory_usage = [df.estimated_size("mb") for df in dataframes.values()]

ax2.bar(range(len(dataset_names)), memory_usage, color=sns.color_palette("viridis", len(dataset_names)))
ax2.set_title('Memory Usage by Dataset', fontsize=14, fontweight='bold')
ax2.set_xlabel('Datasets')
ax2.set_ylabel('Memory Usage (MB)')
ax2.set_xticks(range(len(dataset_names)))
ax2.set_xticklabels(dataset_names, rotation=45, ha='right')

# Add value labels on bars
for i, v in enumerate(memory_usage):
    ax2.text(i, v + max(memory_usage) * 0.01, f'{v:.1f}MB', 
             ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Data quality visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Null values percentage
null_percentages = []
for df in dataframes.values():
    total_cells = df.height * df.width
    null_cells = df.null_count().sum(axis=1)[0]
    null_pct = (null_cells / total_cells) * 100
    null_percentages.append(null_pct)

ax1.bar(range(len(dataset_names)), null_percentages, color='salmon')
ax1.set_title('Data Completeness by Dataset', fontsize=14, fontweight='bold')
ax1.set_xlabel('Datasets')
ax1.set_ylabel('Null Values (%)')
ax1.set_xticks(range(len(dataset_names)))
ax1.set_xticklabels(dataset_names, rotation=45, ha='right')

# Duplicate rows percentage
duplicate_percentages = []
for df in dataframes.values():
    duplicate_rows = df.height - df.unique().height
    duplicate_pct = (duplicate_rows / df.height) * 100
    duplicate_percentages.append(duplicate_pct)

ax2.bar(range(len(dataset_names)), duplicate_percentages, color='lightcoral')
ax2.set_title('Data Uniqueness by Dataset', fontsize=14, fontweight='bold')
ax2.set_xlabel('Datasets')
ax2.set_ylabel('Duplicate Rows (%)')
ax2.set_xticks(range(len(dataset_names)))
ax2.set_xticklabels(dataset_names, rotation=45, ha='right')

plt.tight_layout()
plt.show()

In [ ]:
# Column distribution across datasets
plt.figure(figsize=(12, 8))

# Count columns per dataset
column_counts = [df.width for df in dataframes.values()]

plt.hist(column_counts, bins=min(10, len(set(column_counts))), 
         alpha=0.7, color='skyblue', edgecolor='black')
plt.title('Distribution of Column Counts Across Datasets', fontsize=14, fontweight='bold')
plt.xlabel('Number of Columns')
plt.ylabel('Number of Datasets')
plt.grid(True, alpha=0.3)

# Add statistics
mean_cols = np.mean(column_counts)
median_cols = np.median(column_counts)
plt.axvline(mean_cols, color='red', linestyle='--', label=f'Mean: {mean_cols:.1f}')
plt.axvline(median_cols, color='green', linestyle='--', label=f'Median: {median_cols:.1f}')
plt.legend()

plt.tight_layout()
plt.show()

print(f"📊 Column Statistics:")
print(f"   Mean columns per dataset: {mean_cols:.1f}")
print(f"   Median columns per dataset: {median_cols:.1f}")
print(f"   Min columns: {min(column_counts)}")
print(f"   Max columns: {max(column_counts)}")

## 8. Memory Usage Analysis

Analyze memory consumption of the datasets and optimize data types if necessary using Polars' efficient memory management.

In [ ]:
print("💾 MEMORY USAGE ANALYSIS")
print("=" * 50)

total_memory = 0
memory_breakdown = []

for name, df in dataframes.items():
    memory_mb = df.estimated_size("mb")
    memory_per_row = (memory_mb * 1024 * 1024) / df.height  # bytes per row
    memory_per_col = memory_mb / df.width  # MB per column
    
    breakdown = {
        'Dataset': name,
        'Rows': f"{df.height:,}",
        'Columns': df.width,
        'Memory (MB)': f"{memory_mb:.2f}",
        'Bytes/Row': f"{memory_per_row:.1f}",
        'MB/Column': f"{memory_per_col:.2f}"
    }
    memory_breakdown.append(breakdown)
    total_memory += memory_mb

# Display memory breakdown
memory_df = pd.DataFrame(memory_breakdown)
print(memory_df.to_string(index=False))
print(f"\n📊 Total Memory Usage: {total_memory:.2f} MB")
print(f"📊 Average Memory per Dataset: {total_memory/len(dataframes):.2f} MB")

In [ ]:
# Memory optimization analysis
print("\n🚀 MEMORY OPTIMIZATION OPPORTUNITIES")
print("=" * 50)

# Analyze the largest dataset for optimization opportunities
largest_df = max(dataframes.values(), key=lambda x: x.height)
print(f"Analyzing largest dataset: {largest_name}")
print(f"Current memory usage: {largest_df.estimated_size('mb'):.2f} MB")

# Show current data types (all are likely strings due to infer_schema_length=0)
print("\nCurrent data types:")
dtype_counts = {}
for dtype in largest_df.dtypes:
    dtype_str = str(dtype)
    dtype_counts[dtype_str] = dtype_counts.get(dtype_str, 0) + 1

for dtype, count in dtype_counts.items():
    print(f"  {dtype}: {count} columns")

# Sample optimization: try to infer better types for a subset
print("\n💡 Optimization suggestion:")
print("   Since infer_schema_length=0 was used, all columns are likely strings.")
print("   Consider re-reading with infer_schema_length=None for better type inference.")
print("   This could significantly reduce memory usage for numeric columns.")

# Estimate potential savings
string_columns = sum(1 for dtype in largest_df.dtypes if str(dtype) == 'String')
if string_columns > 0:
    print(f"\n📈 Potential optimization:")
    print(f"   {string_columns} string columns could potentially be optimized")
    print(f"   Estimated potential memory reduction: 30-70% (typical for mixed data types)")

In [ ]:
# Summary and recommendations
print("📋 EXPLORATION SUMMARY & RECOMMENDATIONS")
print("=" * 60)

print(f"📊 Data Overview:")
print(f"   • Total datasets: {len(dataframes)}")
print(f"   • Total rows: {sum(df.height for df in dataframes.values()):,}")
print(f"   • Total memory: {sum(df.estimated_size('mb') for df in dataframes.values()):.2f} MB")

print(f"\n🔍 Data Quality:")
avg_completeness = 100 - np.mean([
    (df.null_count().sum(axis=1)[0] / (df.height * df.width)) * 100 
    for df in dataframes.values()
])
print(f"   • Average data completeness: {avg_completeness:.1f}%")

total_duplicates = sum(df.height - df.unique().height for df in dataframes.values())
print(f"   • Total duplicate rows: {total_duplicates:,}")

print(f"\n💡 Next Steps:")
print(f"   1. Re-read data with proper schema inference for better performance")
print(f"   2. Identify and clean duplicate records")
print(f"   3. Handle missing values based on business requirements")
print(f"   4. Merge datasets on common columns if needed")
print(f"   5. Feature engineering for car price prediction")

print(f"\n✅ Exploration completed successfully!")